## Install Packages

In [45]:
try:
    import pymongo
except:
    !pip install pymongo

try:
    import folium
except:
    !pip install folium

     |████████████████████████████████| 94 kB 3.7 MB/s eta 0:00:011
     |████████████████████████████████| 15.4 MB 3.4 MB/s eta 0:00:01


## Conncet to MongoDB

In [10]:
from pymongo import MongoClient
client = MongoClient('mongo:27017', 
                        username='root',
                         password='example')

# client = MongoClient('localhost:27017', username='root',password='example') # In case access from another instance  

## list of all databases

In [11]:
for db in client.list_databases():
    print(db)

{'name': 'admin', 'sizeOnDisk': 163840.0, 'empty': False}
{'name': 'ahmad', 'sizeOnDisk': 73728.0, 'empty': False}
{'name': 'config', 'sizeOnDisk': 61440.0, 'empty': False}
{'name': 'local', 'sizeOnDisk': 73728.0, 'empty': False}


In [12]:
for db in client.list_database_names():
    print(db)

admin
ahmad
config
local


## Create DataBase

In [13]:
db = client['covid19_db2020']

## Create Collection 

In [14]:
cases = db['cases_col_2020']


## Read json file 

In [29]:
!ls data_json |wc -l

35502


In [30]:
!ls data_json | head -n 10

0.json
10000.json
10001.json
10002.json
10003.json
10004.json
10005.json
10006.json
10007.json
10008.json
ls: write error: Broken pipe


In [34]:
import json 
with open('data_json/1.json', 'r') as f:
    file_dict=json.loads(json.load(f))

In [35]:
file_dict

{'Date': '3/8/2020',
 'Country_Region': 'India',
 'Province_State': None,
 'Difference': 0,
 'Prep_Flow_Runtime': '3/24/2020 9:39:03 AM',
 'Latest_Date': '3/23/2020',
 'Case_Type': 'Deaths',
 'Cases': 0,
 'Lat': 21.0,
 'Long': 78.0,
 'Index': 1}

In [36]:
!ls data_json | wc -l

35502


# Push documents to collection  

In [37]:
result = cases.insert_one(file_dict)

# Push all files 

In [38]:
import glob

List=glob.glob('data_json/*.json')

In [39]:
List[0:10]

['data_json/0.json',
 'data_json/1.json',
 'data_json/10.json',
 'data_json/100.json',
 'data_json/1000.json',
 'data_json/10000.json',
 'data_json/10001.json',
 'data_json/10002.json',
 'data_json/10003.json',
 'data_json/10004.json']

In [40]:
len(List)

35502

In [41]:
from tqdm import tqdm
for file in tqdm(List[1:1001]):
    with open(file, 'r') as f:
        file_dict=json.loads(json.load(f))
        
    result = cases.insert_one(file_dict)
    

100%|██████████| 1000/1000 [00:02<00:00, 449.22it/s]


## Return all the documents in a collection in mongodb

In [42]:
cursor = cases.find({})
for document in cursor:
    print(document)

{'_id': ObjectId('607ce2053243c13a8028aabe'), 'Date': '3/8/2020', 'Country_Region': 'India', 'Province_State': None, 'Difference': 0, 'Prep_Flow_Runtime': '3/24/2020 9:39:03 AM', 'Latest_Date': '3/23/2020', 'Case_Type': 'Deaths', 'Cases': 0, 'Lat': 21.0, 'Long': 78.0, 'Index': 1}
{'_id': ObjectId('607ce20d3243c13a8028aabf'), 'Date': '3/8/2020', 'Country_Region': 'India', 'Province_State': None, 'Difference': 0, 'Prep_Flow_Runtime': '3/24/2020 9:39:03 AM', 'Latest_Date': '3/23/2020', 'Case_Type': 'Deaths', 'Cases': 0, 'Lat': 21.0, 'Long': 78.0, 'Index': 1}
{'_id': ObjectId('607ce22c3243c13a8028aac0'), 'Date': '3/8/2020', 'Country_Region': 'India', 'Province_State': None, 'Difference': 0, 'Prep_Flow_Runtime': '3/24/2020 9:39:03 AM', 'Latest_Date': '3/23/2020', 'Case_Type': 'Deaths', 'Cases': 0, 'Lat': 21.0, 'Long': 78.0, 'Index': 1}
{'_id': ObjectId('607ce2353243c13a8028aac1'), 'Date': '3/8/2020', 'Country_Region': 'India', 'Province_State': None, 'Difference': 0, 'Prep_Flow_Runtime': '3

In [43]:
cursor = cases.find({})
Locations=[]
Name=[]
for document in tqdm(cursor):
    #print(document['Lat'],document['Long'],document['Case_Type'])
    Locations.append([document['Lat'],document['Long']])
    Name.append(document['Case_Type'])

1003it [00:00, 109817.45it/s]


In [46]:
import folium
map = folium.Map(location=[38.9, -77.05], zoom_start=8)
folium.Marker([38.9, -77.05], 
                  popup='PSUT is the best'
                  ).add_to(map)
folium.Marker([38.9, -77.2], 
                  popup='PSUT is the best always'
                  ).add_to(map)
map

In [47]:
import folium
map = folium.Map(location=[38.9, -77.05], zoom_start=2)
for point in range(0, len(Name)):
    folium.Marker(Locations[point], 
                  popup=Name[point]
                  ).add_to(map)
map

In [48]:
import folium
map = folium.Map(location=[38.9, -77.05],tiles="Stamen Terrain", zoom_start=2)
for point in range(0, len(Name)):
    folium.Marker(Locations[point], 
                  popup=Name[point]
                  ).add_to(map)
map



In [49]:


import folium
map = folium.Map(location=[38.9, -77.05],tiles="Stamen Toner", zoom_start=2)
for point in range(0, len(Name)):
    folium.Marker(Locations[point], 
                  popup=Name[point]
                  ).add_to(map)
map


In [52]:
map.save('CovideCases.html')

In [53]:
!du -sh data_json

0	data_json
